In [40]:
import pandas as pd
import numpy as np
import acquire
print('Done!')

Done!


In [41]:
df=acquire.load_zillow_data()
df.head()

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,buildingclasstypeid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,parcelid.1,logerror,transactiondate.1,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,buildingclassdesc
0,NaN,NaN,NaN,261.0,NaN,NaN,NaN,14297519,1727539,NaN,...,14297519,0.025595,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
1,NaN,NaN,NaN,261.0,NaN,NaN,NaN,17052889,1387261,NaN,...,17052889,0.055619,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
2,NaN,NaN,NaN,261.0,NaN,NaN,NaN,14186244,11677,NaN,...,14186244,0.005383,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
3,NaN,NaN,NaN,261.0,2.0,NaN,NaN,12177905,2288172,NaN,...,12177905,-0.103410,2017-01-01,NaN,NaN,Central,Single Family Residential,NaN,NaN,NaN
4,NaN,NaN,NaN,266.0,2.0,NaN,1.0,10887214,1970746,NaN,...,10887214,0.006940,2017-01-01,Central,NaN,Central,Condominium,NaN,NaN,NaN


In [42]:
def columns_data(df):
    columns_data = pd.concat([pd.Series(df.isnull().sum()), pd.Series(df.isnull().sum()/df.shape[0])], axis=1)
    columns_data.reset_index(inplace = True)
    columns_data.columns=(['columns', 'num_rows_missing', 'pct_rows_missing'])
    return columns_data

columns_data(df)


,columns,num_rows_missing,pct_rows_missing
0,buildingclasstypeid,77366,0.999806
1,typeconstructiontypeid,77159,0.997131
2,storytypeid,77331,0.999354
3,propertylandusetypeid,0,0.000000
4,heatingorsystemtypeid,27941,0.361083
...,...,...,...
65,heatingorsystemdesc,27941,0.361083
66,propertylandusedesc,0,0.000000
67,storydesc,77331,0.999354
68,typeconstructiondesc,77159,0.997131


In [43]:
counts = pd.DataFrame(df.isnull().sum(axis=1).value_counts().sort_values())
counts.reset_index(inplace = True)
counts.columns = ['num_cols_missing', 'num_rows']
counts['pct_cols_missing'] = counts['num_cols_missing']/df.shape[1]
counts = counts.sort_values('num_cols_missing').reset_index(drop= True)
counts

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.328571
1,24,13,0.342857
2,25,24,0.357143
3,26,65,0.371429
4,27,316,0.385714
5,28,455,0.400000
6,29,5270,0.414286
7,30,3455,0.428571
8,31,9891,0.442857
9,32,12579,0.457143


In [44]:
def summarize(df):
    print(f"INFO \n {df.info} \n")
    print(f"DESCRIPTION\n {df.describe()} \n")
    print(f"SHAPE \n {df.shape} \n")
    
summarize(df)

INFO 
 <bound method DataFrame.info of        buildingclasstypeid  typeconstructiontypeid  storytypeid  \
0                      NaN                     NaN          NaN   
1                      NaN                     NaN          NaN   
2                      NaN                     NaN          NaN   
3                      NaN                     NaN          NaN   
4                      NaN                     NaN          NaN   
...                    ...                     ...          ...   
77376                  NaN                     NaN          NaN   
77377                  NaN                     NaN          NaN   
77378                  NaN                     NaN          NaN   
77379                  NaN                     NaN          NaN   
77380                  NaN                     NaN          NaN   

       propertylandusetypeid  heatingorsystemtypeid  architecturalstyletypeid  \
0                      261.0                    NaN                       N

In [45]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    columns_to_keep = []
    columns = df.columns
    for col in columns:   
        if df[col].notnull().sum()/df.shape[0] >= prop_required_column:
            columns_to_keep.append(col)
    df = df[columns_to_keep]
    df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row
    df = df.loc[df['row_to_keep'] == True]
    df.drop(columns= ['row_to_keep'], inplace = True)
    return df

In [46]:
df1 = handle_missing_values(df, .7, .5)
df1.isnull().sum()

<ipython-input-45-6a752da0c502>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row


propertylandusetypeid              0
parcelid                           0
id                                 0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                609
calculatedfinishedsquarefeet     196
finishedsquarefeet12            3632
fips                               0
fullbathcnt                      609
latitude                           0
longitude                          0
lotsizesquarefeet               8239
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity                    1471
regionidcounty                     0
regionidzip                       50
roomcnt                            0
yearbuilt                        263
structuretaxvaluedollarcnt       112
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              2
taxamount                          5
censustractandblock              244
transactiondate                    0
p

In [47]:
print(df1.shape)
print(df.shape)

(77381, 31)
(77381, 70)


In [48]:
df1.isnull().sum()

propertylandusetypeid              0
parcelid                           0
id                                 0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                609
calculatedfinishedsquarefeet     196
finishedsquarefeet12            3632
fips                               0
fullbathcnt                      609
latitude                           0
longitude                          0
lotsizesquarefeet               8239
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity                    1471
regionidcounty                     0
regionidzip                       50
roomcnt                            0
yearbuilt                        263
structuretaxvaluedollarcnt       112
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              2
taxamount                          5
censustractandblock              244
transactiondate                    0
p

In [49]:
df.columns

Index(['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
       'propertylandusetypeid', 'heatingorsystemtypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'parcelid', 'id',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17',

In [51]:
df.shape

(77381, 70)

In [38]:
df = df[df.propertylandusetypeid.isin([261, 262, 273])]
df.drop(columns = ['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
       'propertylandusetypeid', 'heatingorsystemtypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id', 'unitcnt'], inplace = True)
df.drop(columns = [c for c in df.columns if c.endswith('.1')], inplace = True)
df['has_basement'] = df['basementsqft'] > 0
df.replace({True:1, False:0}, inplace = True)
df